<a href="https://colab.research.google.com/github/medulka/commercial-llms/blob/main/RAG_over_gaia_x_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG search over Gaia-X Documents

A tool how to ask questions over the Gaia-X documents.
Inspired by the showcase of Tobias Oberrauch (KI-Bundesverband): https://github.com/tobiasoberrauch/rag-showcase.

The pdfs of Gaia-x documents are stored in my personal Google Drive and have to be occassionally updated.
In this version, LLM Mistral available from the Huggingface repository was used to create the embeddings. The vector database Qdrant was chosen for the data storage.

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jul  2 09:51:38 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 54.8 gigabytes of available RAM



In [4]:
!pip install langchain
!pip install langchain_qdrant
!pip install langchain_huggingface

!pip install pypdf
!pip install google-colab

!pip install qdrant-client

Import the pdf files from the personal Google drive.

In [14]:
import os
from google.colab import drive
from langchain.document_loaders import PyPDFLoader
from rich import print

# mount a personal Google Drive content
drive.mount("/content/drive")
path = "/content/drive/MyDrive/gaia-x-documents/"

# getting a list of all the fiels in the directory
pdf_files = [f for f in os.listdir(path) if f.endswith('.pdf')]


loaders = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(os.path.join(path, pdf_file))
    loaders.append(loader)

print(f"Loaded {len(loaders)} pdfs")

documents = []
for loader in loaders:
    document = loader.load_and_split()
    documents.append(document)

print(f"Loaded {len(document)} documents")
print("len_documents: ", len(documents))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Loaded 7 pdfs

Loaded 10 documents

len_documents:  7

Creating a collection in the Qdrant database.

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, PointStruct, VectorParams
from langchain_qdrant import Qdrant


collection_name = "gaia-x-documents"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

from google.colab import userdata

os.environ["QDRANT_URL"] = userdata.get('QDRANT_URL')
os.environ["QDRANT_API_KEY"] = userdata.get('QDRANT_API_KEY')

client = QdrantClient(url=userdata.get('QDRANT_URL'), api_key=userdata.get('QDRANT_API_KEY'))
print(client)

if not client.collection_exists(collection_name):
    client.create_collection(collection_name=collection_name, vectors_config=VectorParams(size = 128, distance=Distance.COSINE))
    print("Collection created")
else:
    print("Collection already exists")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when p

Collection already exists


Populating the database.

In [15]:
for i in range(len(documents)):

    vector_store = Qdrant.from_documents(
        url = os.environ["QDRANT_URL"],
        api_key = os.environ["QDRANT_API_KEY"],
        documents=documents[i],
        embedding=embeddings,
        collection_name=collection_name,
        force_recreate = True,
        )

In [16]:
collection_info = client.get_collection(collection_name)
print(collection_info)

CollectionInfo(
    status=<CollectionStatus.GREEN: 'green'>,
    optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>,
    vectors_count=None,
    indexed_vectors_count=0,
    points_count=10,
    segments_count=2,
    config=CollectionConfig(
        params=CollectionParams(
            vectors=VectorParams(
                size=384,
                distance=<Distance.COSINE: 'Cosine'>,
                hnsw_config=None,
                quantization_config=None,
                on_disk=None,
                datatype=None,
                multivector_config=None
            ),
            shard_number=1,
            sharding_method=None,
            replication_factor=1,
            write_consistency_factor=1,
            read_fan_out_factor=None,
            on_disk_payload=True,
            sparse_vectors=None
        ),
        hnsw_config=HnswConfig(
            m=16,
            ef_construct=100,
            full_scan_threshold=10000,
            max_indexing_threads=0,
            on_disk=False,
            payload_m=None
        ),
        optimizer_config=OptimizersConfig(
            deleted_threshold=0.2,
            vacuum_min_vector_number=1000,
            default_segment_number=0,
            max_segment_size=None,
            memmap_threshold=None,
            indexing_threshold=20000,
            flush_interval_sec=5,
            max_optimization_threads=None
        ),
        wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0),
        quantization_config=None
    ),
    payload_schema={}
)

Query the database.

In [18]:
answer = vector_store.similarity_search("What is Trust Anchor?")
print(answer[0].page_content)
print("\npage:", answer[0].metadata["page"])
print("\nsource:", answer[0].metadata["source"])

Attribute Card. Trust
AnchorComment
providedBy 1 State a resolvable link to the participant
self-description providing the service
aggregationO
f[]0..* State a resolvable link to the resources
self-description related to the service
and that can exist independently of it.
dependsOn[] 0..* State a resolvable link to the service
offering self-description related to
the service and that can exist
independently of it.
termsAndCond
itions[]1..* State a resolvable link to the Terms and
Conditions applying to that service.
policy[] 1..* State a list of policy expressed using a DSL
(e.g., Rego or ODRL) (access control,
throttling, usage, retention, …)
dataProtecti
onRegime[]0..* State a list of data protection regime from the
list available below
dataAccountE
xport[]1..* State list of methods to export data from your
user’s account out of the service
termsAndConditions structure
Attribute Card. Trust Anchor Comment
URL 1 State a resolvable link to document
hash 1 State sha256 hash of the above document.
dataAccountExport structure
The purpose is to enable the participant ordering the service to assess the feasibility to export its
personal and non-personal data out of the service.

page: 1

source: /content/drive/MyDrive/gaia-x-documents/Service & Subclasses - Gaia-X Trust Framework - 22.10 Release.pdf

In [19]:
search_result= vector_store.similarity_search_with_score("What is Gaia-X Trust Anchor?")
result, score = search_result[0]
print(result.page_content)
print(f"\nScore: {score}")

A catalogue service is a subclass of serviceOffering used to browse, search, filter services
and resources.
Attribute Card. Trust
AnchorComment
getVerifiableC
redentialsIDs1..* State a route used to synchronize
catalogues and retrieve the list of
Verifiable Credentials (issuer,
id). [1]
[1]: Using the Verifiable Credential terms, it’s up to the issuer to control and decide if the
credential’s id can be dereferenced and the holder to implement access control of the verifiable
credentials.
6.7 Interoperability, Portability, Switchability and Intellectual
Property Protection (experimental)
This section introduces important Gaia-X principles.
While the format of the attributes are not yet defined, the sections below give insights about the
future requirements.
6.7.1 Interoperability
This covers how the service offering can commu nicate with another service, such as making or
responding to requests.
The aim of the future attributes is to provide:
a list of agreements in place with other service providers to ensure interoperability between
systems
a list of URL to interoperability capabilities and transparency documentation
See ISO/IEC 19941:2017-5.2.1 for a description of the facets that need to be addressed.
6.7.2 Data Portability
This refers to the porting of data (structured or unstructured) from one cloud service to another,
public or private.
Note: Data portability will inevitably be limited to Customer Data and a subset of Derived Data as
determined by contract, privacy regulations, etc. See the ServiceOffering.dataAccountExport

Score: 0.48444247

In [13]:
from typing import List
from rich.console import Console
from rich.table import Table
from langchain_core.documents import Document

def create_table(title: str, documents: List[Document]):
    table = Table(title=title, show_lines=True)
    table.add_column("page")
    table.add_column("page_content")

    for document in documents:
        table.add_row(str(document.metadata["page"]), document.page_content)

    return table

similar_documents = vector_store.search("Was ist Gaia-X Trust Anchor?", "similarity")

console = Console()
console.print(create_table("similar_documents", similar_documents))

                                                 similar_documents                                                 
┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ page ┃ page_content                                                                                             ┃
┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20   │ Several Gaia-X Lighthouse projects  are implementing this concept of Data Intermediary, with different   │
│      │ perimeters and operating models adapted to their ecosystem                                               │
│      │ specificities. Accordingly, it was decided to stick here to basic data exchange concepts and to let      │
│      │ Lighthouse projects define higher level concepts like Data Intermediary                                  │
│      │ according to their needs.                                                                                │
│      │ 2.2.6 4.6 Gaia-X Trust Anchors                                                                           │
│      │ Gaia-X Trust Anchors are bodies, parties, i.e., Conformity Assessment Bodies or technical means          │
│      │ accredited  by the Gaia-X Association to be trustworthy anchors in the                                   │
│      │ cryptographic chain of keypairs used to digitally sign statements or claims about an object.             │
│      │ For each accredited Trust Anchor, a specific scope of attestation  is defined.                           │
│      │ The list of valid Trust Anchors and rules is available via the Gaia-X Registry.                          │
│      │ There are cases where the Trust Anchor is relative to another property in a claim.                       │
│      │ Example: In the case of a gx:DataResource , the consent property  must be signed by the gx:Participant   │
│      │ identified by the dataController  property, itself                                                       │
│      │ signed by the gx:Participant  identified by the producedBy  property, itself signed by the credential’s  │
│      │ issuer with a keypair from a chain of certificates with at its                                           │
│      │ root a Gaia-X accredited state Trust Service Provider .2.2.6 4.6 Gaia-X Trust Anchors                    │
│      │ - 21/52 - ©2024 Gaia-X European Association for Data and Cloud AISBL                                     │
├──────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 21   │ Ecosystem Trust Anchors follow the same principle, but are defined by the ecosystem-specific governance. │
│      │ Kroki                                                                                                    │
│      │ In this example, the dataController ’s participant, producedBy ’s participant and the Trust Service      │
│      │ Provider are Trust Anchors because they are mandatory fixed                                              │
│      │ anchors in the credential chain of signatures. In this same example, all the claims could also be signed │
│      │ by the issuer if the dataController ’s participant,                                                      │
│      │ producedBy ’s participant and issuer are the same participant.                                           │
│      │ 4.6.1  Gaia-X Trusted Data sources and Gaia-X Notaries                                                   │
│      │ When an accredited Gaia-X Trust Anchor is not capable of issuing cryptographic material nor signing      │
│      │ claims directly, then the Gaia-X Association accredits one or                                            │
│      │ more Notaries, which will perform a validation  based on objective evidence from a Gaia-X Trusted Data   │
│      │ source and will issue an attestation  about the